In [2]:
import pandas as pd
from transformers import pipeline

def get_sentiment_accuracy(df):
    rows_total = 0
    rows_guessed = 0

    for index in df.index:
        rows_total = rows_total + 1
        label = df["label"][index]
        human_label = df["human_label"][index]
        
        if label == human_label:
            rows_guessed = rows_guessed + 1

    return rows_guessed / rows_total

def applyLabel(text):
    classifier_result = classifier(text, ["positive", "negative", "neutral"], multi_class=True)
    labels_dict = {key: value for key, value in zip(classifier_result['labels'], classifier_result['scores'])}

    # it is not optimal to return max value as sometimes neutral phrase may be false negative or false positive
    if (labels_dict["positive"] >= 0.5):
        return "positive"
    if (labels_dict["negative"] >= 0.4):
        return "negative"
    return "neutral"

classifier = pipeline(
                      task="zero-shot-classification",
                      model="facebook/bart-large-mnli"
                    )
dataset = pd.read_csv("dataset.csv")
dataset['label'] = dataset['text'].apply(applyLabel)
dataset.to_csv("dataset_with_sentiment.csv")

negative_df = dataset.loc[dataset['human_label'] == "negative"]
neutral_df = dataset.loc[dataset['human_label'] == "neutral"]
positive_df = dataset.loc[dataset['human_label'] == "positive"]

negatives_accuracy = get_sentiment_accuracy(negative_df)
neutral_accuracy = get_sentiment_accuracy(neutral_df)
positive_accuracy = get_sentiment_accuracy(positive_df)
overall_accuracy = get_sentiment_accuracy(dataset)

f = open("sentiment_accuracy.txt", "w")
result_string = (
    "overall accuracy: " + str(overall_accuracy) + "\n"
    "negatives accuracy: " + str(negatives_accuracy) + "\n"
    "neutral accuracy: " + str(neutral_accuracy) + "\n"
    "positive_accuracy: " + str(positive_accuracy) 
)
f.write(result_string)
f.close()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `mul